In [29]:
from astropy.io import fits
from astropy import wcs

import numpy as np
from ants import cvMe, fitsPlay

conv = cvMe.convert()
fp = fitsPlay.fitsplay()



home = '/home/'
#home = '/Users/'

rootDir = home+'maccagni/Projects/MFS/FornaxA/MUSE/'
cubeDir = rootDir+'cubes/'

GAL = 'FornaxE'
RUN = '_4650-5500-sv40-v20'

resDir= cubeDir+'results/'+GAL+RUN+'/'

inTable = resDir+GAL+'_table.fits'
#specTable = resDir+GAL+'_AllSpectra.fits'
starTable = resDir+GAL+'_ppxf-bestfit.fits'
waveTable = resDir+GAL+'_VorSpectra_linear.fits'


resOut = resDir+'mycubes/'



outCube = resOut+'Cube.fits'
outStar = resOut+'StarCube.fits'
outLines = resOut+'LineCube.fits'






print '''\t+---------+\n\t Inputs loaded\n\t+---------+''' 

	+---------+
	 Inputs loaded
	+---------+


In [30]:
#----------------------#
# From Fits Table to FITS Image - PPXF
#----------------------#

diffusion = 1e-5

tab = fits.open(inTable)

head = tab[0].header
headTab = tab[1].header
dataTab = tab[1].data


#tabSpec = fits.open(specTable)
#headTabSpec = tabSpec[0].header
#headSpec = tabSpec[1].header


tabStar = fits.open(starTable)
headTabStar = tabStar[0].header
headStar = tabStar[1].header
dataStar = tabStar[1].data

tabWave = fits.open(waveTable)
dataSpec = tabWave[1].data
dataWave = np.array(tabWave[2].data)
dataWave = [item for t in dataWave for item in t] 

print len(dataWave)


print '''\t+---------+\n\t fits saved\n\t+---------+'''     

603
	+---------+
	 fits saved
	+---------+


In [32]:
# these are arbitrary but must correspond, I choose the centre of the galaxy(usually offset with fov)
crPix3 =1

#D
crPix1=523  #to change!!!
crPix2=273  #to changes!!!!
#C
crPix1=693  #to change!!!
crPix2=661

crPix1=200  #to change!!!
crPix2=125


crVal1='03:22:41.734' ## A & D cubes
crVal2='-37:12:28.62'

#crVal1='03:22:45.331' ## B & C cubes
#crVal2='-37:13:55.33'
#crVal1='03:22:47.040'
#crVal2='-37:12:58.14'

crVal3 = np.min(dataWave)
ra = conv.hms2deg(crVal1)
dec = conv.dms2deg(crVal2)

head['CRPIX1'] = crPix1
head['CRPIX2'] = crPix2

xMin = np.min(dataTab['X'])
xMax = np.max(dataTab['X'])

shapeX = (xMax-xMin)/head['PIXSIZE']

yMin = np.min(dataTab['Y'])
yMax = np.max(dataTab['Y'])

shapeY = (yMax-yMin)/head['PIXSIZE']

xAxis = (np.linspace(1, shapeX+1, shapeX+1) - head['CRPIX1']) *head['PIXSIZE']
yAxis = (np.linspace(1, shapeY+1, shapeY+1) - head['CRPIX2']) *head['PIXSIZE']

deltaLambda = (np.max(dataWave)-np.min(dataWave))/len(dataWave)

data=np.empty([len(dataWave),yAxis.shape[0],xAxis.shape[0]])

Stars=np.empty([len(dataWave),yAxis.shape[0],xAxis.shape[0]])

Lines=np.empty([len(dataWave),yAxis.shape[0],xAxis.shape[0]])

w = wcs.WCS(naxis=3)

pixelSize = head['PIXSIZE']/3600.

# Set up an "Airy's zenithal" projection
# Vector properties may be set with Python lists, or Numpy arrays
w.wcs.crpix = [crPix1, crPix2, crPix3]
w.wcs.cdelt = np.array([-pixelSize,pixelSize,deltaLambda])
w.wcs.crval = [ra, dec, crVal3]
w.wcs.ctype = ["RA---TAN", "DEC--TAN",'AWAV']

header = w.to_header()


#w.wcs.set_pv([(2, 1, 45.0)])
header['EQUINOX'] = 2000


hdu = fits.PrimaryHDU(data=data,header=header)


del header['LATPOLE']
del header['LONPOLE']

#IF FORNAX B & C
#>>> hh['CRVAL1'] = 50.673892
#>>> hh['CRVAL2'] = -37.207935
#>>> hh['CRPIX1'] = 630
#>>> hh['CRPIX2'] = 629
#>>> fits.writeto('LineCub


#create AllSpectra datacube
for i in xrange(0,dataTab['ID'].shape[0]):

    #print xAxis
    #print yAxis
    indexX = ((xAxis < (np.round(dataTab['X'][i],1)+diffusion)) & 
              ((np.round(dataTab['X'][i],1)-diffusion) < xAxis))
    indexY = ((yAxis < (np.round(dataTab['Y'][i],1)+diffusion)) & 
              ((np.round(dataTab['Y'][i],1)-diffusion) < yAxis))       

    xx = np.where(indexX)[0]
    yy = np.where(indexY)[0]

    indexBin =  dataTab['BIN_ID'][i]
    if indexBin>0:
    
        tmp = np.array(dataSpec[indexBin][0][:])
        tmp = tmp.tolist()
        data[:,yy[0],xx[0]] = tmp

        tmp = np.array(dataStar[indexBin][1][:-1])
        tmp = tmp.tolist()
        Stars[:,yy[0],xx[0]] = tmp

Lines = np.subtract(data,Stars)
        
fits.writeto(outCube,data,header,overwrite=True)
fits.writeto(outStar,Stars,header,overwrite=True)
fits.writeto(outLines,Lines,header,overwrite=True)

print '''\t+---------+\n\t fits saved\n\t+---------+'''     

/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


	+---------+
	 fits saved
	+---------+


In [33]:
#----------------------#
# Cut around OIII line
#----------------------#

zmin = 150
zmax = 250



oIII = resOut+'OIIICube.fits'''

fp.zCubeCut(outLines,zmin,zmax,oIII)

ff = fits.open(oIII)

hh = ff[0].header
dd = ff[0].data


if 'LATPOLE' in hh:
    del hh['LATPOLE']
if 'LONPOLE' in hh:
    del hh['LONPOLE']
if "RADESYS" in hh:
    del hh['RADESYS']
if 'WCSAXES' in hh:
    del hh['WCSAXES']
if 'CUNIT3' in hh:
    del hh['CUNIT3']

hh['CDELT3'] = hh['CDELT3']
hh['CRVAL3'] = hh['CRVAL3']


fits.writeto(oIII,dd,hh,overwrite=True)

print '''\t+---------+\n\t fits saved\n\t+---------+'''     

4937.23094657
1.2406444486
	+---------+
	 fits saved
	+---------+


In [13]:
#----------------------#
# Convert to VRAD
#----------------------#
C = 2.99792458e8
z = 0.00587
vsys= C*z
#vsys = 1732e3

Orest = 5006.843
Oexp = Orest*(1+0.00587)
print Oexp

oIII = resOut+'OIIICube.fits'''

oIIIVrad= resOut+'OIIICubeVrad.fits'

f = fits.open(oIII)
hh = f[0].header
print hh
dd = f[0].data


vOptFin = ( ((hh['CRVAL3']+(hh['CDELT3']*hh['NAXIS3'])) -Orest) /(Orest))*C+vsys

vOptIn = ((hh['CRVAL3']-Orest)/Orest)*C+vsys

vDelOpt = (vOptFin-vOptIn)/hh['NAXIS3']


vRadFin = vOptFin/(1.+vOptFin/C)

vRadIn = vOptIn/(1.+vOptIn/C)

vDelRad = (vRadFin-vRadIn)/hh['NAXIS3']


hh['CRVAL3'] = 0.0
hh['CDELT3'] = 0.0
hh['CRPIX3'] = 0.0
if 'LATPOLE' in hh: 
    del hh['LATPOLE']
if 'LONPOLE' in hh:
    del hh['LONPOLE']

    
fits.writeto(oIIIVrad,dd,hh,overwrite=True)

print '''\t+---------+\n\t fits saved\n\t+---------+'''     

5036.23316841
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    3 / number of array dimensions                     NAXIS1  =                  318                                                  NAXIS2  =                  318                                                  NAXIS3  =                  100                                                  CRPIX1  =                201.0 / Pixel coordinate of reference point            CRPIX2  =                125.0 / Pixel coordinate of reference point            CRPIX3  =                    1 / Pixel coordinate of reference point            CDELT1  = -5.5555555555556E-05 / [deg] Coordinate increment at reference point  CDELT2  =  5.5555555555556E-05 / [deg] Coordinate increment at reference point  CDELT3  =      1.2406444485959 / [m] Coordinate increment at reference point    CUNIT1  = 'deg'           

In [36]:
#----------------------#
# Cut around Hb line
#----------------------#

zmin = 80
zmax = 100

Hb = resOut+'HbCube.fits'''

fp.zCubeCut(outLines,zmin,zmax,Hb)


print '''\t+---------+\n\t fits saved\n\t+---------+'''     

4849.90019093
1.24064786063
	+---------+
	 fits saved
	+---------+


In [37]:
#----------------------#
# Convert to VRAD
#----------------------#
C = 2.99792458e8
z = 0.00587
vsys= C*z
#vsys = 1732e3

Hbrest = 4861.333
Hbexp = Hbrest*(1+0.00587)

HbVrad= resOut+'HbCubeVrad.fits'

f = fits.open(Hb)
hh = f[0].header
dd = f[0].data


vOptFin = ( ((hh['CRVAL3']+(hh['CDELT3']*hh['NAXIS3'])) -Hbrest) /(Hbrest))*C+vsys

vOptIn = ((hh['CRVAL3']-Hbrest)/Hbrest)*C+vsys

vDelOpt = (vOptFin-vOptIn)/hh['NAXIS3']


vRadFin = vOptFin/(1.+vOptFin/C)

vRadIn = vOptIn/(1.+vOptIn/C)

vDelRad = (vRadFin-vRadIn)/hh['NAXIS3']


hh['CRVAL3'] = vRadIn
hh['CDELT3'] = vDelRad
hh['CTYPE3'] = 'VRAD'

#del head['RADESYS']
if 'LATPOLE' in hh: 
    del hh['LATPOLE']
if 'LONPOLE' in hh:
    del hh['LONPOLE']
#w.wcs.set_pv([(2, 1, 45.0)])
hh['EQUINOX'] = 2000



fits.writeto(HbVrad,dd,hh,overwrite=True)

print '''\t+---------+\n\t fits saved\n\t+---------+'''     


print '''\t+---------+\n\t fits saved\n\t+---------+'''     

	+---------+
	 fits saved
	+---------+
	+---------+
	 fits saved
	+---------+
